In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"Key exists and starts with {openai_api_key[:10]}")
else:
    print('There is an issue with the API Key or it might not exist')

MODEL = 'gpt-4.1-mini'
openai = OpenAI()

Key exists and starts with sk-proj-61


In [4]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
def chat(message, history):
    history = [{'role':h ['role'], 'content':h ['content']} for h in history]
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

In [6]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [7]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for {destination_city}")
    price = ticket_prices.get(destination_city.lower(), 'unnknown ticket price')
    return f"The price of a ticket to {destination_city} is {price}"

In [8]:
get_ticket_price('London')

Tool called for London


'The price of a ticket to London is $799'

In [9]:
price_function = {
    'name': 'get_ticket_price',
    'description': 'Get the price of a return ticket to the destination city.',
    'parameters': {
        'type': 'object',
        'properties': {
            'destination_city': {
                'type': 'string',
                'description': 'The city that the customer wants to travel to',
            },
        },
        'required': ['destination_city'],
        'additionalProperties': False
    }
}

In [10]:
tools = [{"type": "function", "function": price_function}]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
def chat(message, history):
    history = [{'role':h ['role'], 'content':h ['content']} for h in history]
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].finish_reason
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool called for London
